# Week 3: Edit distance, assembly, and overlaps



We're going to use **dynamic programming** to find edit distances and approximate matches that account for insertions and deletions. It'll also allow us to find similar substrings between strings. Toward the end of this week, we'll begin to discuss genome assembly.

### Recursive edit distance

This algorithm is ridiculously slow. It runs on O(n!) time.

In [21]:
def edit_dist_recursive(a, b):
    if len(a) == 0:
        return len(b)
    if len(b) == 0:
        return len(a)
    delt = 1 if a[-1] != b[-1] else 0
    
    return min(edit_dist_recursive(a[:-1], b[:-1])+delt, 
               edit_dist_recursive(a[:-1], b)+1, 
               edit_dist_recursive(a, b[:-1])+1)

As we can see, it takes a long time.

In [24]:
%%time
edit_dist_recursive('shake spea', 'Shakespear')

CPU times: total: 3.05 s
Wall time: 3.04 s


3

### Implementing dynamic programming for edit distance

Dynamic programming is much faster.

In [132]:
def edit_dist(x, y):
    # initialize empty x+1 by y+1 matrix
    D = [[0]*(len(y)+1) for i in range(len(x)+1)]

    # first column and row should be 0, 1, 2, 3...
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i


    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            dist_hor = D[i][j-1] + 1 # value of cell to left + 1 penalty
            dist_ver = D[i-1][j] + 1 # value of cell above + 1 penalty
            if x[i-1] == y[j-1]:
                dist_diag = D[i-1][j-1] # if characters match then no penalty
            else:
                dist_diag = D[i-1][j-1] + 1 # else +1 penalty
            D[i][j] = min(dist_hor, dist_ver, dist_diag)
            
    return D[-1][-1] # return very bottom right value

In [31]:
%%time
edit_dist('shake spea', 'Shakespear')

CPU times: total: 0 ns
Wall time: 0 ns


3

### Global alignment

We want to penalize certain kinds of errors more than others. Transitions are much more frequent and more likely due to chance mutation rather than transversions, which are more significant.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8a/All_transitions_and_transversions.svg/1200px-All_transitions_and_transversions.svg.png" width=400px />

And skips are really bad. I think we're using the Needleman-Wunsch algorithm.

In [82]:
alphabet = ['A', 'C', 'G', 'T']
score = [[0, 4, 2, 4, 8],  # A>A:0, A>C:4, A>G:2, A>T:4, A>null: 8
         [4, 0, 4, 2, 8],
         [2, 4, 0, 4, 8],
         [4, 2, 4, 0, 8],
         [8, 8, 8, 8, 8]] 

In [131]:
def global_alignment(x, y):
    # initialize empty x+1 by y+1 matrix
    D = [[0]*(len(y)+1) for i in range(len(x)+1)]

    for i in range(1, len(x)+1): # don't want to touch the top left char
        D[i][0] = D[i-1][0] + score[alphabet.index(x[i-1])][-1] 
    for i in range(1, len(y)+1):
        D[0][i] = D[0][i-1] + score[-1][alphabet.index(y[i-1])] 

    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            dist_hor = D[i][j-1] + score[-1][alphabet.index(y[j-1])]
            dist_ver = D[i-1][j] + score[alphabet.index(x[i-1])][-1]
            if x[i-1] == y[j-1]:
                dist_diag = D[i-1][j-1] # if characters match then no penalty
            else:
                dist_diag = D[i-1][j-1] + score[alphabet.index(x[i-1])][alphabet.index(y[j-1])]

            print(dist_hor, dist_ver, dist_diag)
            D[i][j] = min(dist_hor, dist_ver, dist_diag)
            
    return D[-1][-1] # return very bottom right value

In [88]:
global_alignment('A', 'AA')

16 16 0
8 24 8


8

## Overlaps between pairs of reads

In [117]:
def overlap(a, b, min_length=0):
    '''length of longest suffix of a matching prefix of b'''
    start = 0
    while True:
        start = a.find(b[:min_length], start)
        if start == -1:
            return 0
        if b.startswith(a[start:]):
            return len(a)-start # length of overlap
        start += 1

In [118]:
overlap("CC","ACC") # pay close attention to min_length

0

## Naive Overlap Map

In [119]:
from itertools import permutations

In [120]:
list(permutations([1, 2, 3], 2))

[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

In [121]:
def naive_overlap_map(reads, k):
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            olaps[(a,b)] = olen
    return olaps

In [122]:
reads = ['ACGGATGATC', 'GATCAAGT', 'TTCACGGA']
naive_overlap_map(reads, 3)

{('ACGGATGATC', 'GATCAAGT'): 4, ('TTCACGGA', 'ACGGATGATC'): 5}

## Homework

We saw how to adapt dynamic programming to find approximate occurrences of a pattern in a text. Recall that:

* Rows of the dynamic programming matrix are labeled with bases from P and columns with bases from T
* Elements in the first row are set to 0
* Elements in the first column are set to 0, 1, 2, ..., as for edit distance
* Other elements are set in the same way as elements of a standard edit distance matrix
* The minimal value in the bottom row is the edit distance of the closest match between P and T

First, download the provided excerpt of human chromosome 1

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

Second, parse it using the readGenome function we wrote before.

In [124]:
def read_genome(filename):
    f = open(filename, "r")
    genome = f.read().split("\n")[1:]
    f.close()
    return "".join(genome)

In [125]:
chromosome = read_genome('genomes/chr1.GRCh38.excerpt.fasta')

Third, adapt the editDistance function we saw in practical (copied below) to answer questions 1 and 2 below. Your function should take arguments p (pattern), t (text) and should return the edit distance of the match between P and T with the fewest edits.

In [ ]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

Hint: In the "A new solution to approximate matching" video we saw that the best approximate match of P=GCGTATGC within T=TATTGGCTATACGGTThad 2 edits. You can use this and other small examples to double-check that your function is working.

### Q1. 

What is the edit distance of the best match between pattern GCTGATCGATCGTACG and the excerpt of human chromosome 1?  (Don't consider reverse complements.)


    Rows of the dynamic programming matrix are labeled with bases from P and columns with bases from T
    Elements in the first row are set to 0
    Elements in the first column are set to 0, 1, 2, ..., as for edit distance
    Other elements are set in the same way as elements of a standard edit distance matrix
    The minimal value in the bottom row is the edit distance of the closest match between P and T


In [164]:
def approximate_edit_distance(p, t):
    D = [[0]*(len(t)+1) for i in range(len(p)+1)]

    # leave first row set to 0
    
    for i in range(len(p)+1):
        D[i][0] = i # first column are set to 0, 1, 2 ... for edit distance

    
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            dist_hor = D[i][j-1] +1
            dist_ver = D[i-1][j] +1
            if p[i-1] == t[j-1]:
                dist_drag = D[i-1][j-1]
            else:
                dist_drag = D[i-1][j-1] + 1
            D[i][j] = min(dist_hor, dist_ver, dist_drag)

    return min(D[len(p)])

In [166]:
p = 'GCTGATCGATCGTACG'
approximate_edit_distance(p, chromosome)

3

### Q2.

What is the edit distance of the best match between pattern `GATTTACCAGATTGAG` and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [169]:
p = 'GATTTACCAGATTGAG'
approximate_edit_distance(p, chromosome)

2

### Q3. 

In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match) between two strings. The function is copied below.

In [ ]:
def overlap(a, b, min_length=0):
    '''length of longest suffix of a matching prefix of b'''
    start = 0
    while True:
        start = a.find(b[:min_length], start)
        if start == -1:
            return 0
        if b.startswith(a[start:]):
            return len(a)-start # length of overlap
        start += 1

Say we are concerned only with overlaps that (a) are exact matches (no differences allowed), and (b) are at least `k` bases long. To make an overlap graph, we could call `overlap(a, b, min_length=k)` on every possible pair of reads from the dataset.  Unfortunately, that will be very slow!

Consider this: Say we are using k=6, and we have a read `a` whose length-6 suffix is `GTCCTA`.  Say `GTCCTA` does not occur in any other read in the dataset.  In other words, the 6-mer `GTCCTA` occurs at the end of read `a` and nowhere else.  It follows that `a`'s suffix cannot possibly overlap the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read `a` and a prefix of some other read, we can ignore any reads that don't contain the length-k suffix of `a`.  This is good news because it can save us a lot of work!

Here is a suggestion for how to implement this idea. You don't have to do it this way, but this might help you. Let every k-mer in the dataset have an associated Python `set` object, which starts out empty.  We use a Python dictionary to associate each k-mer with its corresponding set.

1) For every k-mer in a read, we add the read to the `set` object corresponding to that k-mer. If our read is `GATTA` and k=3, we would add `GATTA` to the set objects for GAT, ATT and TTA.  We do this for every read so that, at the end, each set contains all reads containing the corresponding k-mer.
2) Now, for each read `a`, we find all overlaps involving a suffix of `a`.  To do this, we take `a`'s length-k suffix, find all reads containing that k-mer (obtained from the corresponding setset) and call overlap(a, b, min_length=k) for each.

The most important point is that we do not call `overlap(a, b, min_length=k)` if `b` does not contain the length-k suffix of `a`.

Download and parse the read sequences from the provided Phi-X FASTQ file. We'll just use their base sequences, so you can ignore read names and base qualities.  Also, no two reads in the FASTQ have the same sequence of bases.  This makes things simpler.

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match.  Ignore reverse complements.

Hints 
1. Your function should not take much more than 15 seconds to run on this 10,000-read dataset, and maybe much less than that.  (Our solution takes about 3 seconds.) If your function is much slower, there is a problem somewhere.
2. Remember not to overlap a read with itself. If you do, your answers will be too high.
3. You can test your implementation by making up small examples, then checking that (a) your implementation runs quickly, and (b) you get the same answer as if you had simply called `overlap(a, b, min_length=k)` on every pair of reads.  We also have provided a couple examples you can check against.

In [197]:
def read_fastq(filename):
    with open(filename, 'r') as f:
        content = f.read().rstrip("\n").split("\n")
        seqs = [content[i] for i in range(len(content)) if i%4 == 1]
        quals = [content[i] for i in range(len(content)) if i%4 == 3]
    return seqs, quals

In [198]:
seqs, _ = read_fastq('genomes/ERR266411_1.for_asm.fastq')

In [211]:
def make_kmers(string, k):
    return set([string[i:i+k] for i in range(len(string)-k+1)])
        
def find_overlaps(reads, min_length):
    kmer_read_dict = {}
    for read in reads:
        kmers = make_kmers(read, min_length)
        for kmer in kmers:
            if kmer not in kmer_read_dict.keys():
                kmer_read_dict[kmer] = set()
            kmer_read_dict[kmer].add(read)

    overlaps = []
    for read in reads:
        suff = read[-min_length:]
        found_reads = kmer_read_dict[suff]
        for r in found_reads:
            if overlap(read, r) >= 30 and read != r:
                overlaps.append((read, r))
    return overlaps
        

Picture the overlap graph corresponding to the overlaps just calculated.  How many edges are in the graph?  In other words, how many distinct pairs of reads overlap?

In [212]:
olaps = find_overlaps(seqs, 30)

In [213]:
len(olaps)

904746

## Q4
Picture the overlap graph corresponding to the overlaps computed for the previous question. How many nodes in this graph have at least one outgoing edge?  (In other words, how many reads have a suffix involved in an overlap?)

In [218]:
srcs = set([a[0] for a in olaps])
len(srcs)

7161